In [22]:
from tensorflow.keras import utils

path = './Data'
batch_size = 64

train = utils.image_dataset_from_directory(
    directory=path + '/Train',
    shuffle = True,
    seed = 1,
    image_size = (28,28),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    directory=path + '/Test',
    shuffle = True,
    seed = 1,
    image_size = (28,28),
    batch_size = batch_size
)

Found 27455 files belonging to 24 classes.
Found 7172 files belonging to 24 classes.


In [23]:
from tensorflow.keras import models, layers, activations, initializers

model = models.Sequential([
    layers.Input(shape=(28,28,3)), 
    
    layers.Resizing(28,28),
    layers.Rescaling(1./255),
    layers.RandomRotation((-0.2, 0.2)),

    layers.Conv2D(10, kernel_size=(3,3), strides=(1,1), padding='same'),
    layers.Conv2D(5, kernel_size=(6,6), strides=(1,1), padding='same'),

    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),

    layers.Dense(64, activation=activations.elu, kernel_initializer=initializers.RandomNormal()),

    layers.Dense(64, activation=activations.gelu, kernel_initializer=initializers.RandomNormal()),

    layers.Dense(64, activation=activations.sigmoid, kernel_initializer=initializers.RandomNormal()),

    layers.Dropout(0.2),

    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),
    
    layers.Dense(24, activation=activations.softmax, kernel_initializer=initializers.RandomNormal())
])

from tensorflow.keras import optimizers, losses, metrics

lr = 0.0001

model.compile(
    optimizer = optimizers.Adam(
        learning_rate = lr    
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [ metrics.sparse_categorical_accuracy ]
)

from tensorflow.keras import callbacks

patience = 5
epochs = 100
model_path = "./model.keras"

model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    verbose = True,
    
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks .ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Epoch 1/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - loss: 3.1632 - sparse_categorical_accuracy: 0.0511 - val_loss: 3.0985 - val_sparse_categorical_accuracy: 0.0616
Epoch 2/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - loss: 2.9122 - sparse_categorical_accuracy: 0.1038 - val_loss: 2.7711 - val_sparse_categorical_accuracy: 0.1011
Epoch 3/100
109/429 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 2.7869 - sparse_categorical_accuracy: 0.1287

KeyboardInterrupt: 